<a href="https://colab.research.google.com/github/dcpatton/GenerativeAI/blob/main/DLA_AgenticRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install llama_index==0.12.16 llama-index-experimental==0.5.3 -q

In [2]:
import os
import getpass
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.experimental.query_engine import PandasQueryEngine
from typing import Tuple
from geopy.distance import geodesic
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.bridge.pydantic import BaseModel
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopy.geocoders import Nominatim
import time
from llama_index.core.tools import FunctionTool
from llama_index.agent.openai import OpenAIAgent
import unittest
import json
from typing import Optional, Required
from llama_index.core.llms import ChatMessage
import warnings
warnings.filterwarnings('ignore')
from openai import OpenAI as OAI

assert pd.__version__ == '2.2.2'
assert gpd.__version__ == '1.0.1'
assert json.__version__ == '2.0.9'

In [3]:
# setup OpenAI api_key

if not os.environ.get('OPENAI_API_KEY'):
  os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter the OpenAI API Key(which starts with sk-): ")

Enter the OpenAI API Key(which starts with sk-): ··········


In [4]:
Settings.llm = OpenAI(temperature=0.0, model="gpt-3.5-turbo")

## Data

Inspired by our "Pedals to the Metal" sessions https://youtu.be/y0gsfJ2pTOs

Capital Bikeshare https://capitalbikeshare.com/

Our source of data is bike trips in January 2025: https://s3.amazonaws.com/capitalbikeshare-data/202501-capitalbikeshare-tripdata.zip

In [5]:
!wget https://s3.amazonaws.com/capitalbikeshare-data/202501-capitalbikeshare-tripdata.zip
!unzip 202501-capitalbikeshare-tripdata.zip

--2025-04-03 17:43:09--  https://s3.amazonaws.com/capitalbikeshare-data/202501-capitalbikeshare-tripdata.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 3.5.24.38, 52.217.114.248, 52.216.204.5, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|3.5.24.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11453201 (11M) [application/zip]
Saving to: ‘202501-capitalbikeshare-tripdata.zip’

202501-capitalbikes 100%[===================>]  10.92M  11.5MB/s    in 0.9s    

2025-04-03 17:43:10 (11.5 MB/s) - ‘202501-capitalbikeshare-tripdata.zip’ saved [11453201/11453201]

Archive:  202501-capitalbikeshare-tripdata.zip
  inflating: 202501-capitalbikeshare-tripdata.csv  
  inflating: __MACOSX/._202501-capitalbikeshare-tripdata.csv  


In [45]:
data = pd.read_csv('./202501-capitalbikeshare-tripdata.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285731 entries, 0 to 285730
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             285731 non-null  object 
 1   rideable_type       285731 non-null  object 
 2   started_at          285731 non-null  object 
 3   ended_at            285731 non-null  object 
 4   start_station_name  197609 non-null  object 
 5   start_station_id    197609 non-null  float64
 6   end_station_name    194709 non-null  object 
 7   end_station_id      194514 non-null  float64
 8   start_lat           285731 non-null  float64
 9   start_lng           285731 non-null  float64
 10  end_lat             285548 non-null  float64
 11  end_lng             285548 non-null  float64
 12  member_casual       285731 non-null  object 
dtypes: float64(6), object(7)
memory usage: 28.3+ MB


In [46]:
# eliminate trips that don't start and stop at a station
data = data[data['start_station_id'].notna()]
data = data[data['end_station_id'].notna()]

In [47]:
# remove unused columns
data = data.drop(['ride_id', 'rideable_type', 'started_at', 'ended_at', 'member_casual'], axis='columns').drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49633 entries, 0 to 283766
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_station_name  49633 non-null  object 
 1   start_station_id    49633 non-null  float64
 2   end_station_name    49633 non-null  object 
 3   end_station_id      49633 non-null  float64
 4   start_lat           49633 non-null  float64
 5   start_lng           49633 non-null  float64
 6   end_lat             49633 non-null  float64
 7   end_lng             49633 non-null  float64
dtypes: float64(6), object(2)
memory usage: 3.4+ MB


In [48]:
# verify that all stations are present in start_stations
l1 = data.start_station_id.unique().tolist()
data = data[data['end_station_id'].isin(l1)]
l1 = data.end_station_id.unique().tolist()
data = data[data['start_station_id'].isin(l1)]
assert data.start_station_id.nunique() == data.end_station_id.nunique()
assert set(data.start_station_id.unique()) == set(data.end_station_id.unique())

In [49]:
# data cleaning
data['start_station_name'] = data['start_station_name'].str.strip()

In [50]:
# reduce to minimum amount of station data we need
data = (data[['start_station_name', 'start_station_id', 'start_lat', 'start_lng']]
        .rename(columns={'start_station_name':'station_name', 'start_station_id':'station_id', 'start_lat':'latitude', 'start_lng':'longitude'
                        })
        .drop_duplicates()
      )
data['station_id'] = data['station_id'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9378 entries, 0 to 272343
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_name  9378 non-null   object 
 1   station_id    9378 non-null   int64  
 2   latitude      9378 non-null   float64
 3   longitude     9378 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 366.3+ KB


In [51]:
# add a very special new station
data.loc[len(data)] = ['DLA', 99999, 39.003313, -77.104553]

In [52]:
# convert to geopandas
# https://geopandas.org/

geometry = [Point(xy) for xy in zip(data['longitude'], data['latitude'])]
gdf = gpd.GeoDataFrame(data, geometry=geometry, crs='EPSG:4326')
gdf = gdf.drop(['latitude', 'longitude'], axis='columns').drop_duplicates()
gdf.sample(5)

,station_name,station_id,geometry
388,14th & Quincy St NW,31401,POINT (-77.03269 38.93802)
129972,23rd & M St NW,31128,POINT (-77.05037 38.90531)
157779,5th & K St NW,31600,POINT (-77.01922 38.90276)
170112,Lincoln Park / 13th & East Capitol St NE,31619,POINT (-76.98841 38.89049)
115447,Tanner Park,31533,POINT (-77.00213 38.91136)


## How well does a chatbot do alone

In [53]:
prompt = """I want to find a route using CaBi to start at the closest station to
Pike & Rose, North Bethesda MD 20852 and ending at the closest station to
10 Center Dr, Bethesda, MD 20814.
Can you create one for me?"""

client = OAI()

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
)
print(chat_completion.choices[0].message.content)

Sure! Here is a suggested route using Capital Bikeshare (CaBi) from Pike & Rose to 10 Center Dr:

1. Start at the Pike & Rose CaBi station located at:
11580 Old Georgetown Rd, North Bethesda, MD 20852

2. Ride south on Old Georgetown Rd towards Rockville Pike.

3. Turn left onto Rockville Pike and continue south.

4. Turn right onto Center Dr and continue towards 10 Center Dr, Bethesda, MD 20814.

5. End at the Bethesda Metro CaBi station located at:
7450 Wisconsin Ave, Bethesda, MD 20814

This route is approximately 2 miles and should take about 15-20 minutes to bike. Enjoy your ride!


## Build tools

* use osm map tool to find location to start at, and end at
* then make tool to get closest cabi start and end locations.
* build with geopandas.

In [54]:
# tool for getting lat, lon given an address
# https://nominatim.org/

def get_latlon(address):
  geolocator = Nominatim(user_agent='DLA_app')
  try:
    time.sleep(1)
    location = geolocator.geocode(address)
    print(location)
    return location.latitude, location.longitude
  except Exception as e:
        print(f"Error geocoding address: {str(e)}")
        return None

test_address = "1600 Pennsylvania Avenue NW, Washington, DC 20500"

lat, lon = get_latlon(test_address)
assert lat == 38.897699700000004
assert lon == -77.03655315

location_tool = FunctionTool.from_defaults(fn=get_latlon
                                           , name='get_location'
                                           , description='Gets latitude and longitude of a location given the address')

White House, 1600, Pennsylvania Avenue Northwest, Penn Quarter, Ward 2, Washington, District of Columbia, 20500, United States


In [55]:
# test location_tool

test_agent = OpenAIAgent.from_tools(
    [location_tool]
    , verbose=True
)


response = test_agent.chat('''
  What is the latitude of 1600 Pennsylvania Avenue NW, Washington, DC 20500?
  Respond with a JSON object.
          ''', tool_choice='auto')
d = json.loads(response.response)
assert 38.897699700000004 == d['latitude']


response = test_agent.chat('''
  What is the longitude of 1600 Pennsylvania Avenue NW, Washington, DC 20500?
  Respond with a JSON object.
          ''', tool_choice='auto')
d = json.loads(response.response)
assert -77.03655315 == d['longitude']


Added user message to memory: 
  What is the latitude of 1600 Pennsylvania Avenue NW, Washington, DC 20500?
  Respond with a JSON object.
          
=== Calling Function ===
Calling function: get_location with args: {"address":"1600 Pennsylvania Avenue NW, Washington, DC 20500"}
White House, 1600, Pennsylvania Avenue Northwest, Penn Quarter, Ward 2, Washington, District of Columbia, 20500, United States
Got output: (38.897699700000004, -77.03655315)

Added user message to memory: 
  What is the longitude of 1600 Pennsylvania Avenue NW, Washington, DC 20500?
  Respond with a JSON object.
          


In [56]:
# tool for querying CaBi geopandas dataframe
# https://docs.llamaindex.ai/en/stable/examples/query_engine/pandas_query_engine/

query_engine = PandasQueryEngine(df=gdf, verbose=True)

response = query_engine.query("""
What is the geometry of the Oak Leaf & Lockwood	station?
"""
)


> Pandas Instructions:
```
df[df['station_name'] == 'Oak Leaf & Lockwood'].geometry.values[0]
```
> Pandas Output: POINT (-76.9945391 39.0379809)


In [57]:
data[data['station_name']=='Oak Leaf & Lockwood']

,station_name,station_id,latitude,longitude
55562,Oak Leaf & Lockwood,32094,39.037981,-76.994539


In [58]:
response = query_engine.query("""
What is the latitude of the Fort Stanton Rec Center station?
""")

> Pandas Instructions:
```
df[df['station_name'] == 'Fort Stanton Rec Center']['geometry'].iloc[0].y
```
> Pandas Output: 38.85736903758752


In [59]:
data[data['station_name']=='Fort Stanton Rec Center']['latitude']

,latitude
27648,38.857369


In [60]:
#Can it do something more sophisticated?
response = query_engine.query("""
What is the nearest station to latitude = 39.036 and longitude = -76.9951 ?
""")

#Nope. Though it looks almost like it is almost using the right logic.

> Pandas Instructions:
```
df['distance'] = df['geometry'].apply(lambda x: x.distance(Point(-76.9951, 39.036)))
df.loc[df['distance'].idxmin()]['station_name']
```
> Pandas Output: There was an error running the output as Python code. Error message: name 'Point' is not defined


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/llama_index/experimental/query_engine/pandas/output_parser.py", line 42, in default_output_processor
    safe_exec(ast.unparse(module), {}, local_vars)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/llama_index/experimental/exec_utils.py", line 171, in safe_exec
    return exec(__source, _get_restricted_globals(__globals), __locals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "/usr/local/lib/python3.11/dist-packages/geopandas/geoseries.py", line 752, in apply
    result = super().apply(func, args=args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/series.py", line 4924, in apply
    ).apply()
      ^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/apply.py", line 1427, i

In [61]:
# build a tool using GeoPandas methods to find the nearest station

def find_nearest_point(latitude, longitude):

    target_point = Point(longitude, latitude)  # Note: longitude comes first in Point()
    gdf['distance'] = gdf.geometry.distance(target_point)

    # Find the row with minimum distance
    nearest_point = gdf.loc[gdf['distance'].idxmin()]

    return nearest_point['station_id'], nearest_point['station_name']

assert find_nearest_point(39.036, -76.9951) == (32094, 'Oak Leaf & Lockwood')

nearest_station_tool = FunctionTool.from_defaults(fn=find_nearest_point
                                           , name='nearest_station_tool'
                                           , description='Gets the nearest station given a latitude and longitude as input')

In [62]:
# tool to look up latlon via station name
def get_lat_lon_by_station_name(name: str) -> Tuple[float, float]:
  lat = query_engine.query('What is the latitude of ' + name)
  lon = query_engine.query('What is the longitude of ' + name)
  return lat, lon


station_location_tool = FunctionTool.from_defaults(fn=get_lat_lon_by_station_name
                                           , name='get_station_location'
                                           , description='Gets latitude and longitude of a station via the station name')

# tool to calculate distance in miles using geodisic class from geopy
# https://geopy.readthedocs.io/en/stable/#module-geopy.distance
def get_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
  loc1 = (lat1, lon1)
  loc2 = (lat2, lon2)
  distance = geodesic(loc1, loc2).miles
  return distance

distance_tool = FunctionTool.from_defaults(fn=get_distance
                                           , name='distance'
                                           , description='Gets distance in miles given the latitude and longitude of two locations')

In [63]:
test_agent = OpenAIAgent.from_tools(tools=[station_location_tool, distance_tool, nearest_station_tool], verbose=True)

In [64]:
response = test_agent.chat("""
  What is the latitude of the station named Fort Stanton Rec Center?
""")

Added user message to memory: 
  What is the latitude of the station named Fort Stanton Rec Center?

=== Calling Function ===
Calling function: get_station_location with args: {"name":"Fort Stanton Rec Center"}
> Pandas Instructions:
```
df[df['station_name'] == 'Fort Stanton Rec Center']['geometry'].values[0].y
```
> Pandas Output: 38.85736903758752
> Pandas Instructions:
```
df[df['station_name'] == 'Fort Stanton Rec Center']['geometry'].x.iloc[0]
```
> Pandas Output: -76.97768801437633
Got output: (Response(response='38.85736903758752', source_nodes=[], metadata={'pandas_instruction_str': "df[df['station_name'] == 'Fort Stanton Rec Center']['geometry'].values[0].y", 'raw_pandas_output': '38.85736903758752'}), Response(response='-76.97768801437633', source_nodes=[], metadata={'pandas_instruction_str': "df[df['station_name'] == 'Fort Stanton Rec Center']['geometry'].x.iloc[0]", 'raw_pandas_output': '-76.97768801437633'}))



In [65]:
print(response.response)

The latitude of the station named Fort Stanton Rec Center is 38.85736903758752.


In [66]:
data[data['station_name']=='Southern Ave Metro']['latitude']

,latitude
26045,38.840107


In [67]:
response = test_agent.chat("""
  What is id and name of the nearest station to latitude of 38.840, longitude of -76.975?
""")

Added user message to memory: 
  What is id and name of the nearest station to latitude of 38.840, longitude of -76.975?

=== Calling Function ===
Calling function: nearest_station_tool with args: {"latitude":38.84,"longitude":-76.975}
Got output: (np.int64(32409), 'Southern Ave Metro')



In [68]:
response.response

'The nearest station to the latitude 38.840 and longitude -76.975 is with ID 32409 and named Southern Ave Metro.'

In [69]:
response = test_agent.chat("""
What is the distance between the station named Southern Ave Metro station and the station named Jones Branch & Westbranch Dr?
""")

Added user message to memory: 
What is the distance between the station named Southern Ave Metro station and the station named Jones Branch & Westbranch Dr?

=== Calling Function ===
Calling function: get_station_location with args: {"name": "Southern Ave Metro"}
> Pandas Instructions:
```
df[df['station_name'] == 'Southern Ave Metro']['geometry'].values[0].y
```
> Pandas Output: 38.840107
> Pandas Instructions:
```
df[df['station_name'] == 'Southern Ave Metro']['geometry'].x.iloc[0]
```
> Pandas Output: -76.9753
Got output: (Response(response='38.840107', source_nodes=[], metadata={'pandas_instruction_str': "df[df['station_name'] == 'Southern Ave Metro']['geometry'].values[0].y", 'raw_pandas_output': '38.840107'}), Response(response='-76.9753', source_nodes=[], metadata={'pandas_instruction_str': "df[df['station_name'] == 'Southern Ave Metro']['geometry'].x.iloc[0]", 'raw_pandas_output': '-76.9753'}))

=== Calling Function ===
Calling function: get_station_location with args: {"name":

In [70]:
print(response.response)

The distance between the station named Southern Ave Metro and the station named Jones Branch & Westbranch Dr is approximately 14.60 miles.


## Route object

In [71]:
# a collection of bikeshare reservations
routes = {}

# define a Route class representing a bikeshare use and tools for working with Routes
class Route(BaseModel):
  start_station_name: Optional[str] = None
  end_station_name: Optional[str] = None
  start_station_id: Optional[str] = None
  end_station_id: Optional[str] = None
  start_station_latitude: Optional[float] = None
  start_station_longitude: Optional[float] = None
  end_station_latitude: Optional[float] = None
  end_station_longitude: Optional[float] = None
  distance: Optional[float] = None

def create_route(start_station_id: str, end_station_id: str, start_station_name: str, end_station_name: str) -> str:
  key = start_station_id + '_to_' + start_station_id
  routes[key] = Route()
  setattr(routes[key], 'start_station_id', start_station_id)
  setattr(routes[key], 'end_station_id', end_station_id)
  setattr(routes[key], 'start_station_name', start_station_name)
  setattr(routes[key], 'end_station_name', end_station_name)
  return """Route for """ + key + """ created."""

def update_route(key: str, property1: str, value: float) -> str:
    route = routes[key]
    setattr(route, property1, value)
    return f"Route {key} updated with {property1} = {value}"

def get_route_state(key: str) -> str:
    return str(routes[key].dict())


create_route_tool = FunctionTool.from_defaults(fn=create_route
                                               , name='create_route'
                                               , description='Creates a Route object and adds it to the routes dictionary')
update_route_tool = FunctionTool.from_defaults(fn=update_route
                                               , name='update_route'
                                               , description='updates a Route objects properties given the key')
get_route_state_tool = FunctionTool.from_defaults(fn=get_route_state
                                                  , name='get_route_state'
                                                  , description='gets the state of the routes properties given the key')

## Main agent

In [72]:
prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"""You are a helpful assistent who is connected to the route system and helping with making a Route.
            The Route will have information about the stations and the distance between them.
            The user will provide you with the starting address and ending address.
            You will need to do the following:

            1. Get the latitude and longitude of the location of the starting address.
            2. Get the latitude and longitude of the location of the ending address.
            3. Find the id and station name for the station that is the nearest to the starting address location.
            4. Find the id and station name the station that is the nearest to the ending address location.
            5. Determine the distance of the route.
            6. Confirm that the route is acceptable to the User.
            7. If the user accepts the route, then Create a Route object with the start_station_id, start_station_name, end_station_id, end_station_name.
            8. Update the Route object with the distance.

            """
        ),
    )
]

In [73]:
main_agent = OpenAIAgent.from_tools(
            tools=[
                location_tool, station_location_tool, distance_tool
                , nearest_station_tool, create_route_tool, update_route_tool
                , get_route_state_tool
            ],
            prefix_messages=prefix_messages,
            # max_function_calls=100,
            # allow_parallel_tool_calls=False,
            verbose=True,
        )

## Agent Chat

In [74]:
response = main_agent.chat("""
I want to find a route using CaBi between two addresses. Can you help me please
""")

Added user message to memory: 
I want to find a route using CaBi between two addresses. Can you help me please



In [75]:
print(response.response)

Of course! I can assist you in finding a route using Capital Bikeshare (CaBi) between two addresses. Please provide me with the starting address and the ending address for the route.


In [76]:
response = main_agent.chat("""
I want to start at Pike & Rose, North Bethesda MD 20852 and end at 10 Center Dr, Bethesda, MD 20814
""")

Added user message to memory: 
I want to start at Pike & Rose, North Bethesda MD 20852 and end at 10 Center Dr, Bethesda, MD 20814

=== Calling Function ===
Calling function: get_location with args: {"address":"Pike & Rose, North Bethesda MD 20852"}
Pike & Rose, North Bethesda, Montgomery County, Maryland, United States
Got output: (39.0507958, -77.11700105906954)

=== Calling Function ===
Calling function: get_location with args: {"address":"10 Center Dr, Bethesda, MD 20814"}
NIH Clinical Center, 10, Center Drive, Battery Lane, Pooks Hill, East Bethesda, Montgomery County, Maryland, 20892, United States
Got output: (39.0016448, -77.10454223658931)

=== Calling Function ===
Calling function: nearest_station_tool with args: {"latitude": 39.0507958, "longitude": -77.11700105906954}
Got output: (np.int64(32083), 'Rockville Pike & Rose Ave')

=== Calling Function ===
Calling function: nearest_station_tool with args: {"latitude": 39.0016448, "longitude": -77.10454223658931}
Got output: (np.

In [77]:
print(response.response)

The distance between the starting station (Rockville Pike & Rose Ave) and the ending station (DLA) is approximately 3.46 miles.

Would you like to proceed with this route?


In [78]:
# check to see if the reservation has been made
routes

{}

In [79]:
response = main_agent.chat('Yes please')

Added user message to memory: Yes please
=== Calling Function ===
Calling function: create_route with args: {"start_station_id":"32083","end_station_id":"99999","start_station_name":"Rockville Pike & Rose Ave","end_station_name":"DLA"}
Got output: Route for 32083_to_32083 created.



In [80]:
print(response.response)

The route has been successfully created from Rockville Pike & Rose Ave to DLA with a distance of 3.46 miles. 

If you need any more assistance or information, feel free to ask!


In [81]:
# check to see if our reservation is completed.
routes

{'32083_to_32083': Route(start_station_name='Rockville Pike & Rose Ave', end_station_name='DLA', start_station_id='32083', end_station_id='99999', start_station_latitude=None, start_station_longitude=None, end_station_latitude=None, end_station_longitude=None, distance=None)}

In [82]:
data.station_name.unique()

array(['8th & F St NE', 'Massachusetts Ave & Dupont Circle NW',
       'Columbia Rd & Belmont St NW', 'New Hampshire Ave & 24th St NW',
       "L'Enfant Plaza / 7th & C St SW", '2nd St & Massachusetts Ave NE',
       '11th & O St NW', '7th & K St NE', '17th & Corcoran St NW',
       '17th & K St NW', '4th & D St NW / Judiciary Square',
       '8th & V St NW', '3rd & H St NW', '16th & Irving St NW',
       'Eisenhower Ave & Mill Race Ln', '5th & K St NW',
       'Mt Vernon Trail & S. Washington St.', 'Fenton St & Ellsworth Dr',
       'N Veitch St & 20th St N', '16th & Harvard St NW',
       '10th & U St NW', 'Fairfax Dr & N Randolph St', '17th & G St NW',
       'Langston Blvd & N Cleveland St', '5th St & Massachusetts Ave NW',
       'Long Bridge Aquatic Center', '15th & L St NW',
       'Columbus Circle / Union Station', '8th & O St NW',
       '18th & L St NW', '14th & Irving St NW',
       'New Hampshire Ave & Ward Pl NW', '15th St & Massachusetts Ave SE',
       '18th St & Pennsyl